# **Topic Modeling**
##(3) LDA Topic Modeling with Gensim
#### The main source that we performed for our project.

# 0. Road datasets


In [ ]:
# Import and Install the necessary libraries
# pip install wordcloud matplotlib

import warnings
warnings.filterwarnings('ignore')


In [ ]:
import pandas as pd

data = pd.read_csv('/Users/leahshin/Documents/myProject/hotel_review_newregion.csv')

In [ ]:
data.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Tags_list,name,region,sub_region
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,8/3/17,7.7,Hotel Arena,Russia,I am so angry that i made this post available ...,397,1403,...,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Russia,Europe,Eastern Europe
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,8/3/17,7.7,Hotel Arena,Ireland,NaN,0,1403,...,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Ireland,Europe,Northern Europe
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/31/17,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficult...,42,1403,...,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,"[' Leisure trip ', ' Family with young childre...",Australia,Oceania,Australia and New Zealand
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/31/17,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk bar...,210,1403,...,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",United Kingdom,Europe,Northern Europe
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/24/17,7.7,Hotel Arena,New Zealand,You When I booked with your company on line yo...,140,1403,...,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",New Zealand,Oceania,Australia and New Zealand


In [ ]:
len(set(data["Reviewer_Nationality"]))

226

In [ ]:
set(data.region)

{'Africa', 'Americas', 'Antarctica', 'Asia', 'Europe', 'Oceania'}

In [ ]:
set(data.sub_region)

{'Antarctica',
 'Australia and New Zealand',
 'Central Asia',
 'Eastern Africa',
 'Eastern Asia',
 'Eastern Europe',
 'Latin America and the Caribbean',
 'Melanesia',
 'Micronesia',
 'Middle Africa',
 'Northern Africa',
 'Northern America',
 'Northern Europe',
 'Polynesia',
 'South-eastern Asia',
 'Southern Africa',
 'Southern Asia',
 'Southern Europe',
 'Western Africa',
 'Western Asia',
 'Western Europe'}

In [ ]:
# check all columns
data.columns.values

array(['Unnamed: 0', 'Hotel_Address', 'Additional_Number_of_Scoring',
       'Review_Date', 'Average_Score', 'Hotel_Name',
       'Reviewer_Nationality', 'Negative_Review',
       'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
       'Positive_Review', 'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'Tags', 'days_since_review', 'lat', 'lng', 'Tags_list', 'name',
       'region', 'sub_region'], dtype=object)

In [ ]:
# Merge both reviews into one column
data['Merged_Review'] = data['Negative_Review'] + " " + data['Positive_Review']

# 1. Text Preprocessing

In [ ]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Define a function to preprocess text (including sentimental words removal)

stop_words = set(stopwords.words('english'))
stop_words_nouns = ["hotel", "place", "room", "5", "4", "3", "2", "1", "wa", "u", "one", "two", "everything", "nothing", "thing"]
stop_words_verbs = ["enjoy", "would", "like", "could", "get", "stay", "visit", "go", "think", "make"]
stop_words_adverbs = ["highly", "within", "really", "well", "even", "though", "bit", "little", "extremely",
                 "definitely", "much", "also", "always", "never", "often", "usually"]
stop_words_adjectives =["great", "positive", "negative", "good", "nice", "lovely", "fantastic", "excellent",
                 "amazing", "wonderful", "horrible", "terrible", "disappointing", "awful", "poor", "super"]

# Not Included: "spacious", "friendly", "helpful", "close", "staff", "experience"

# Merge all custom stopwords lists
all_stopwords = list(stop_words.union(set(stop_words_nouns)))
all_stopwords += stop_words_verbs + stop_words_adverbs + stop_words_adjectives

# Convert the custom stopwords list to a set for faster membership checking
all_stopwords = set(all_stopwords)


In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
        # Remove punctuation and convert to lowercase
        text = text.translate(str.maketrans('', '', string.punctuation)).lower()
        # Tokenize the text
        tokens = word_tokenize(text)

        # Lemmatize the tokens
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Remove stopwords
        filtered_tokens = [word for word in lemmatized_tokens if word not in all_stopwords]
        return filtered_tokens
    else:
        # Handle non-string values (e.g., float) gracefully by returning an empty list
        return []

# Apply text preprocessing to the merged reviews
data['Merged_Review'] = data['Merged_Review'].apply(preprocess_text)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
data['Merged_Review'][0:8]

0    [angry, made, post, available, via, possible, ...
1                                                   []
2    [elderly, difficult, story, narrow, step, ask,...
3    [dirty, afraid, walk, barefoot, floor, looked,...
4    [booked, company, line, showed, picture, thoug...
5    [backyard, total, mess, happen, star, restaura...
6    [cleaner, change, sheet, duvet, everyday, made...
7    [apart, price, brekfast, location, set, park, ...
Name: Merged_Review, dtype: object

<br><br>

# 2. Topic Modeling with Gensim

In [ ]:
import gensim
from gensim import corpora
from gensim import models
from gensim.models import LdaModel
from gensim.models import TfidfModel
from gensim.models import CoherenceModel

In [ ]:
### Create a Dictionary and Corpus:
from gensim import corpora

# Create a dictionary mapping of words to numerical IDs
dictionary = corpora.Dictionary(data['Merged_Review'])

# Create a corpus, which is a list of bag-of-words representations of documents
corpus = [dictionary.doc2bow(doc) for doc in data['Merged_Review']]


#############################################

## Modeling with 10 topics
This code will train an LDA model with 10 topics and print the top 50 words for each topic.


In [ ]:
# Train the LDA model with 10 topics
lda_model_10_topics = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=42, passes=10)

In [ ]:
# Print the topics and their top words
topics = lda_model_10_topics.print_topics(num_words=50)
for topic in topics:
    print(topic)

(0, '0.037*"station" + 0.035*"close" + 0.028*"location" + 0.026*"metro" + 0.025*"city" + 0.024*"walk" + 0.016*"minute" + 0.016*"breakfast" + 0.015*"clean" + 0.015*"near" + 0.014*"center" + 0.014*"easy" + 0.014*"centre" + 0.013*"train" + 0.013*"central" + 0.012*"far" + 0.012*"walking" + 0.012*"restaurant" + 0.011*"area" + 0.010*"tram" + 0.010*"min" + 0.010*"away" + 0.009*"distance" + 0.009*"milan" + 0.009*"around" + 0.008*"stop" + 0.008*"10" + 0.008*"bus" + 0.007*"comfortable" + 0.007*"street" + 0.007*"parking" + 0.007*"right" + 0.007*"access" + 0.007*"tube" + 0.007*"park" + 0.007*"quiet" + 0.007*"located" + 0.006*"public" + 0.006*"main" + 0.006*"nearby" + 0.006*"shopping" + 0.006*"big" + 0.006*"underground" + 0.006*"quite" + 0.006*"transport" + 0.006*"car" + 0.006*"duomo" + 0.005*"airport" + 0.005*"next" + 0.005*"convenient"')
(1, '0.127*"bed" + 0.076*"location" + 0.068*"breakfast" + 0.059*"small" + 0.046*"comfortable" + 0.037*"clean" + 0.033*"pool" + 0.028*"comfy" + 0.025*"size" + 0.0

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Make sure you have the preprocessed and tokenized text data
# This should be a list of lists of tokens
tokenized_texts = data['Merged_Review'].tolist()

# Create a CoherenceModel instance
coherence_model_lda = CoherenceModel(model=lda_model_10_topics, texts=tokenized_texts, dictionary=dictionary, coherence='c_v')

# Calculate the coherence score
coherence_lda = coherence_model_lda.get_coherence()

# Print the average coherence score
print('Average Topic Coherence: %.4f' % coherence_lda)


Average Topic Coherence: 0.7519


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Generate the LDAvis visualization
lda_display = gensimvis.prepare(lda_model_10_topics, corpus, dictionary)

# Save the LDAvis visualization as an HTML file
pyLDAvis.save_html(lda_display, 'lda_visualization.html')

<br><br>



# 3-1-1. Assign each topics to the review
Now, we will assign topics to each review based on the LDA model's predictions.

### 1. Label the topics

In [ ]:
topic_labels = {
    0: "Location",
    1: "Room Conditions",
    2: "Facilities",
    3: "Meals",
    4: "Customer Services",
    5: "Cleanliness",
    6: "Friendly Staff",
    7: "Value",
    8: "Easy Process",
    9: "Noise and Comfort"
}

for idx, topic in lda_model_10_topics.print_topics():
    print(f"Topic {idx}: {topic_labels[idx]} - {topic}")

Topic 0: Location - 0.037*"station" + 0.035*"close" + 0.028*"location" + 0.026*"metro" + 0.025*"city" + 0.024*"walk" + 0.016*"minute" + 0.016*"breakfast" + 0.015*"clean" + 0.015*"near"
Topic 1: Room Conditions - 0.127*"bed" + 0.076*"location" + 0.068*"breakfast" + 0.059*"small" + 0.046*"comfortable" + 0.037*"clean" + 0.033*"pool" + 0.028*"comfy" + 0.025*"size" + 0.023*"bathroom"
Topic 2: Facilities - 0.065*"coffee" + 0.045*"free" + 0.042*"facility" + 0.040*"tea" + 0.029*"spa" + 0.021*"wifi" + 0.019*"water" + 0.016*"bar" + 0.016*"machine" + 0.015*"luggage"
Topic 3: Meals - 0.068*"breakfast" + 0.021*"bar" + 0.021*"food" + 0.018*"restaurant" + 0.015*"service" + 0.012*"choice" + 0.011*"area" + 0.009*"drink" + 0.009*"buffet" + 0.007*"quality"
Topic 4: Customer Services - 0.026*"staff" + 0.025*"day" + 0.024*"time" + 0.022*"check" + 0.018*"reception" + 0.015*"night" + 0.012*"service" + 0.011*"back" + 0.011*"asked" + 0.010*"first"
Topic 5: Cleanliness - 0.055*"shower" + 0.042*"bathroom" + 0.03

<br>

### 2. Get the most relavent topics for each reviews and label it with the most top relevent

In [ ]:
# Function to get the most relevant topic for each document
def get_most_relevant_topic(lda_model_10_topics, corpus):
    most_relevant_topics = []
    for doc in corpus:
        topics = lda_model_10_topics.get_document_topics(doc, per_word_topics=True)
        dominant_topic = max(topics[0], key=lambda x: x[1])[0]
        most_relevant_topics.append(dominant_topic)
    return most_relevant_topics

# Get the most relevant topic for each document
data['Top_Topic'] = get_most_relevant_topic(lda_model_10_topics, corpus)

# Map topic names to topic numbers using the topic_labels dictionary
data['Top_Topic_Label'] = data['Top_Topic'].map(topic_labels)


In [ ]:
data.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,days_since_review,lat,lng,Tags_list,name,region,sub_region,Merged_Review,Top_Topic,Top_Topic_Label
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,8/3/17,7.7,Hotel Arena,Russia,I am so angry that i made this post available ...,397,1403,...,0 days,52.360576,4.915968,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Russia,Europe,Eastern Europe,"[angry, made, post, available, via, possible, ...",9,Noise and Comfort
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,8/3/17,7.7,Hotel Arena,Ireland,NaN,0,1403,...,0 days,52.360576,4.915968,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Ireland,Europe,Northern Europe,[],0,Location
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/31/17,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficult...,42,1403,...,3 days,52.360576,4.915968,"[' Leisure trip ', ' Family with young childre...",Australia,Oceania,Australia and New Zealand,"[elderly, difficult, story, narrow, step, ask,...",2,Facilities
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/31/17,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk bar...,210,1403,...,3 days,52.360576,4.915968,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",United Kingdom,Europe,Northern Europe,"[dirty, afraid, walk, barefoot, floor, looked,...",4,Customer Services
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/24/17,7.7,Hotel Arena,New Zealand,You When I booked with your company on line yo...,140,1403,...,10 days,52.360576,4.915968,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",New Zealand,Oceania,Australia and New Zealand,"[booked, company, line, showed, picture, thoug...",8,Easy Process


In [ ]:
data[["Merged_Review", "Top_Topics", "Top_Topics_Labels"]].head(10)

,Merged_Review,Top_Topics,Top_Topics_Labels
0,"[angry, made, post, available, via, possible, ...","[(9, 0.29813027), (8, 0.24832857), (4, 0.22751...","[Noise and Comfort, Easy Process, Customer Ser..."
1,[],"[(0, 0.1), (1, 0.1), (2, 0.1), (3, 0.1), (4, 0...","[Location, Room Conditions, Facilities, Meals,..."
2,"[elderly, difficult, story, narrow, step, ask,...","[(2, 0.415281), (7, 0.20591322), (9, 0.0935646...","[Facilities, Value, Noise and Comfort, Meals, ..."
3,"[dirty, afraid, walk, barefoot, floor, looked,...","[(4, 0.38528657), (9, 0.19621842), (5, 0.18289...","[Customer Services, Noise and Comfort, Cleanli..."
4,"[booked, company, line, showed, picture, thoug...","[(8, 0.61148167), (4, 0.24193136), (3, 0.09421...","[Easy Process, Customer Services, Meals, Noise..."
5,"[backyard, total, mess, happen, star, restaura...","[(0, 0.28715238), (7, 0.2293834), (6, 0.218429...","[Location, Value, Friendly Staff, Meals, Custo..."
6,"[cleaner, change, sheet, duvet, everyday, made...","[(1, 0.27602145), (4, 0.20604055), (2, 0.20507...","[Room Conditions, Customer Services, Facilitie..."
7,"[apart, price, brekfast, location, set, park, ...","[(3, 0.48896974), (6, 0.2711778), (0, 0.189831...","[Meals, Friendly Staff, Location]"
8,[],"[(0, 0.1), (1, 0.1), (2, 0.1), (3, 0.1), (4, 0...","[Location, Room Conditions, Facilities, Meals,..."
9,"[aircondition, noise, hard, sleep, night, big,...","[(9, 0.37672615), (1, 0.2406975), (3, 0.187581...","[Noise and Comfort, Room Conditions, Meals, Lo..."


<br>

### 3. For top N topics and assign as a list to each row of data

In [ ]:
# Create a function to get the top N topics with probabilities for each document
def get_top_topics(lda_model_10_topics, corpus, num_topics=5):
    top_topics = []
    for doc in corpus:
        topics = lda_model_10_topics.get_document_topics(doc, per_word_topics=True)
        topics_sorted = sorted(topics[0], key=lambda x: x[1], reverse=True)[:num_topics]
        top_topics.append([(topic[0], topic[1]) for topic in topics_sorted])
    return top_topics

# Get the top 10 topics with probabilities for each document
data['Top_Topics'] = get_top_topics(lda_model_10_topics, corpus, num_topics=5)

# Map topic names to topic numbers using the topic_labels dictionary created earlier
data['Top_Topics_Labels'] = data['Top_Topics'].apply(lambda topics: [topic_labels[topic[0]] for topic in topics])


In [ ]:
data.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,lng,Tags_list,name,region,sub_region,Merged_Review,Top_Topic,Top_Topic_Label,Top_Topics,Top_Topics_Labels
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,8/3/17,7.7,Hotel Arena,Russia,I am so angry that i made this post available ...,397,1403,...,4.915968,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Russia,Europe,Eastern Europe,"[angry, made, post, available, via, possible, ...",9,Noise and Comfort,"[(9, 0.29813027), (8, 0.24832857), (4, 0.22751...","[Noise and Comfort, Easy Process, Customer Ser..."
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,8/3/17,7.7,Hotel Arena,Ireland,NaN,0,1403,...,4.915968,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Ireland,Europe,Northern Europe,[],0,Location,"[(0, 0.1), (1, 0.1), (2, 0.1), (3, 0.1), (4, 0...","[Location, Room Conditions, Facilities, Meals,..."
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/31/17,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficult...,42,1403,...,4.915968,"[' Leisure trip ', ' Family with young childre...",Australia,Oceania,Australia and New Zealand,"[elderly, difficult, story, narrow, step, ask,...",2,Facilities,"[(2, 0.415281), (7, 0.20591322), (9, 0.0935646...","[Facilities, Value, Noise and Comfort, Meals, ..."
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/31/17,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk bar...,210,1403,...,4.915968,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",United Kingdom,Europe,Northern Europe,"[dirty, afraid, walk, barefoot, floor, looked,...",4,Customer Services,"[(4, 0.38528657), (9, 0.19621842), (5, 0.18289...","[Customer Services, Noise and Comfort, Cleanli..."
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/24/17,7.7,Hotel Arena,New Zealand,You When I booked with your company on line yo...,140,1403,...,4.915968,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",New Zealand,Oceania,Australia and New Zealand,"[booked, company, line, showed, picture, thoug...",8,Easy Process,"[(8, 0.61148167), (4, 0.24193136), (3, 0.09421...","[Easy Process, Customer Services, Meals, Noise..."


<br><br>

# 3-1-2. Weighted Score Metric System

In [ ]:
region_list = data["region"].unique().tolist()
for region in region_list:
    region_data = data[data['region'] == region]

    # Create a dictionary for scoring
    score_dict = {}

    # Calculate the scores for each topic in the region
    for topics_list in region_data['Top_Topics_Labels']:
        for i, topic in enumerate(topics_list):
            if topic not in score_dict:
                score_dict[topic] = 0
            score_dict[topic] += (10 - i)  # Score calculation

    # Sort the scores and find the rank of topics
    sorted_topics = sorted(score_dict.items(), key=lambda x: (x[1], x[0]), reverse=True)[:10]

    # Output the top five topics for the region
    print(f"The ranking of topics for {region}:")
    for topic, score in sorted_topics:
        print(f"{topic}: {score}")
    print("\n")


The ranking of topics for Europe:
Room Conditions: 2189160
Location: 2138533
Meals: 2099555
Customer Services: 1692857
Facilities: 1504675
Friendly Staff: 1212083
Value: 807449
Noise and Comfort: 800112
Cleanliness: 789772
Easy Process: 585791


The ranking of topics for Oceania:
Location: 165038
Room Conditions: 150102
Meals: 143036
Customer Services: 115161
Facilities: 111653
Friendly Staff: 86730
Value: 54903
Noise and Comfort: 52077
Cleanliness: 51099
Easy Process: 33553


The ranking of topics for Americas:
Location: 322332
Room Conditions: 283325
Meals: 283244
Customer Services: 233745
Facilities: 206563
Friendly Staff: 164163
Value: 112936
Noise and Comfort: 101092
Cleanliness: 99054
Easy Process: 65254


The ranking of topics for Asia:
Location: 464677
Room Conditions: 459292
Meals: 372479
Customer Services: 343986
Facilities: 307797
Friendly Staff: 269386
Value: 229165
Cleanliness: 146883
Noise and Comfort: 127554
Easy Process: 106698


The ranking of topics for Africa:
Locati

In [ ]:
sub_region_list = data["sub_region"].unique().tolist()
for sub_region in sub_region_list:
    sub_region_data = data[data['sub_region'] == sub_region]

    # Create a dictionary for scoring
    score_dict = {}

    # Calculate the scores for each topic in the sub-region
    for topics_list in sub_region_data['Top_Topics_Labels']:
        for i, topic in enumerate(topics_list):
            if topic not in score_dict:
                score_dict[topic] = 0
            score_dict[topic] += (10 - i)  # Score calculation

    # Sort the scores and find the rank of topics
    sorted_topics = sorted(score_dict.items(), key=lambda x: (x[1], x[0]), reverse=True)[:10]

    # Output the top five topics for the sub-region
    print(f"The Ranking of Topics for {sub_region}:")
    for topic, score in sorted_topics:
        print(f"{topic}: {score}")
    print("\n")


The Ranking of Topics for Eastern Europe:
Location: 135081
Room Conditions: 120070
Meals: 109507
Customer Services: 92012
Facilities: 83807
Value: 53462
Friendly Staff: 50924
Cleanliness: 41924
Noise and Comfort: 39642
Easy Process: 29568


The Ranking of Topics for Northern Europe:
Room Conditions: 1666896
Meals: 1619256
Location: 1578786
Customer Services: 1298540
Facilities: 1146019
Friendly Staff: 963683
Noise and Comfort: 622130
Cleanliness: 611113
Value: 576525
Easy Process: 459869


The Ranking of Topics for Australia and New Zealand:
Location: 164124
Room Conditions: 149192
Meals: 142177
Customer Services: 114501
Facilities: 111106
Friendly Staff: 86214
Value: 54500
Noise and Comfort: 51727
Cleanliness: 50699
Easy Process: 33315


The Ranking of Topics for Western Europe:
Location: 276355
Room Conditions: 257868
Meals: 245297
Customer Services: 197969
Facilities: 177823
Friendly Staff: 130406
Value: 116497
Noise and Comfort: 95062
Cleanliness: 89621
Easy Process: 63007


The Ra

<br><br>



# 3-2-1. Apply to the Negative Reviews

In [ ]:
data['Negative_Review_2'] = data['Negative_Review'].apply(preprocess_text)

In [ ]:
data['Tokenized_Negative_Review'] = data['Negative_Review'].apply(lambda doc: word_tokenize(str(doc).lower()))
dictionary2 = corpora.Dictionary(data['Tokenized_Negative_Review'])
corpus2 = [dictionary2.doc2bow(doc) for doc in data['Tokenized_Negative_Review']]

# Function to get the top 7 topics with probabilities for each document
def get_top_topics(lda_model, corpus, num_topics=7):
    top_topics = []
    for doc in corpus:
        topics = lda_model.get_document_topics(doc)
        topics_sorted = sorted(topics, key=lambda x: x[1], reverse=True)[:num_topics]
        top_topics.append([(topic[0], topic[1]) for topic in topics_sorted])
    return top_topics

# Assuming lda_model_10_topics is your trained LDA model
data['Top_Topics_Neg'] = get_top_topics(lda_model_10_topics, corpus2)

# Assuming topic_labels_2 is your dictionary mapping topic numbers to labels
data['Top_Topics_Labels_Neg'] = data['Top_Topics_Neg'].apply(lambda topics: [topic_labels.get(topic[0], "Unknown Topic") for topic in topics])

In [ ]:
region_list = data["region"].unique().tolist()
for region in region_list:
    region_data = data[data['region'] == region]

    # Create a dictionary for scoring
    score_dict_2 = {}

    # Calculate the scores for each topic in the region
    for topics_list in region_data['Top_Topics_Labels_Neg']:
        for i, topic in enumerate(topics_list):
            if topic not in score_dict_2:
                score_dict_2[topic] = 0
            score_dict_2[topic] += (7 - i)  # Adjusted score calculation for top seven topics

    # Sort the scores and find the top seven topics
    sorted_topics = sorted(score_dict_2.items(), key=lambda x: (x[1], x[0]), reverse=True)[:7]

    # Output the top seven topics for the region
    print(f"Top seven topics for {region}:")
    for topic, score in sorted_topics:
        print(f"{topic}: {score}")
    print("\n")


Top seven topics for Europe:
Noise and Comfort: 1304592
Meals: 1247297
Easy Process: 1217705
Value: 1209354
Customer Services: 1205969
Location: 1135833
Facilities: 877992


Top seven topics for Oceania:
Easy Process: 90275
Value: 87512
Meals: 83978
Noise and Comfort: 83350
Location: 83177
Customer Services: 82189
Facilities: 62050


Top seven topics for Americas:
Value: 173850
Easy Process: 170949
Meals: 166762
Noise and Comfort: 166318
Customer Services: 160880
Location: 159305
Facilities: 118685


Top seven topics for Asia:
Meals: 263986
Noise and Comfort: 256614
Easy Process: 255017
Location: 242920
Value: 235029
Customer Services: 216459
Facilities: 188522


Top seven topics for Africa:
Easy Process: 31978
Noise and Comfort: 31778
Meals: 31562
Location: 30427
Value: 29970
Customer Services: 27630
Facilities: 23087


Top seven topics for Antarctica:
Easy Process: 16
Location: 15
Value: 14
Room Conditions: 10
Meals: 10
Noise and Comfort: 7
Facilities: 7




In [ ]:
sub_region_list = data["sub_region"].unique().tolist()
for sub_region in sub_region_list:
    sub_region_data = data[data['sub_region'] == sub_region]

    # Create a dictionary for scoring
    score_dict_2 = {}

    # Calculate the scores for each topic in the sub-region
    for topics_list in sub_region_data['Top_Topics_Labels_Neg']:
        for i, topic in enumerate(topics_list):
            if topic not in score_dict_2:
                score_dict_2[topic] = 0
            score_dict_2[topic] += (7 - i)  # Adjusted score calculation for top seven topics

    # Sort the scores and find the top seven topics
    sorted_topics = sorted(score_dict_2.items(), key=lambda x: (x[1], x[0]), reverse=True)[:7]

    # Output the top seven topics for the sub-region
    print(f"Top seven topics for {sub_region}:")
    for topic, score in sorted_topics:
        print(f"{topic}: {score}")
    print("\n")


Top seven topics for Eastern Europe:
Noise and Comfort: 71721
Meals: 70087
Value: 69292
Easy Process: 67240
Location: 62665
Customer Services: 62216
Facilities: 48441


Top seven topics for Northern Europe:
Noise and Comfort: 991541
Meals: 947509
Customer Services: 935464
Easy Process: 928596
Value: 914838
Location: 864775
Facilities: 670488


Top seven topics for Australia and New Zealand:
Easy Process: 89796
Value: 87023
Meals: 83465
Noise and Comfort: 82790
Location: 82680
Customer Services: 81678
Facilities: 61668


Top seven topics for Western Europe:
Noise and Comfort: 160250
Meals: 150671
Value: 146400
Easy Process: 144203
Customer Services: 138569
Location: 136162
Facilities: 102439


Top seven topics for Southern Europe:
Noise and Comfort: 81080
Meals: 79030
Value: 78824
Easy Process: 77666
Location: 72231
Customer Services: 69720
Facilities: 56624


Top seven topics for Northern America:
Value: 154899
Easy Process: 152942
Noise and Comfort: 148991
Meals: 148819
Customer Servi

In [ ]:
data[["Negative_Review_2", "Top_Topics_Neg", "Top_Topics_Labels_Neg"]].head(10)

,Negative_Review_2,Top_Topics_Neg,Top_Topics_Labels_Neg
0,"[angry, made, post, available, via, possible, ...","[(9, 0.30675742), (4, 0.2258605), (3, 0.097307...","[Noise and Comfort, Customer Services, Meals, ..."
1,[],"[(7, 0.5499771), (8, 0.05001126), (0, 0.050001...","[Value, Easy Process, Location, Room Condition..."
2,"[elderly, difficult, story, narrow, step, ask,...","[(9, 0.40442112), (4, 0.25689125), (0, 0.09865...","[Noise and Comfort, Customer Services, Locatio..."
3,"[dirty, afraid, walk, barefoot, floor, looked,...","[(4, 0.25055608), (9, 0.24485967), (8, 0.13539...","[Customer Services, Noise and Comfort, Easy Pr..."
4,"[booked, company, line, showed, picture, thoug...","[(3, 0.32847464), (9, 0.19419841), (4, 0.18155...","[Meals, Noise and Comfort, Customer Services, ..."
5,"[backyard, total, mess, happen, star]","[(4, 0.49579525), (9, 0.22788627), (7, 0.08638...","[Customer Services, Noise and Comfort, Value, ..."
6,"[cleaner, change, sheet, duvet, everyday, made...","[(3, 0.37577936), (9, 0.3146134), (8, 0.151607...","[Meals, Noise and Comfort, Easy Process, Room ..."
7,"[apart, price, brekfast]","[(9, 0.5585725), (3, 0.18862244), (2, 0.182787...","[Noise and Comfort, Meals, Facilities, Friendl..."
8,"[picture, show, clean, actual, quit, dirty, ou...","[(9, 0.28636014), (8, 0.27139696), (5, 0.13922...","[Noise and Comfort, Easy Process, Cleanliness,..."
9,"[aircondition, noise, hard, sleep, night]","[(6, 0.23211722), (9, 0.20946416), (5, 0.19399...","[Friendly Staff, Noise and Comfort, Cleanlines..."


<br><br>
# 3-2-2. Apply to the Positive Reviews

In [ ]:
data['Positive_Review_2'] = data['Positive_Review'].apply(preprocess_text)

In [ ]:
# Tokenize the 'Positive_Review' column
data['Tokenized_Positive_Review'] = data['Positive_Review'].apply(lambda doc: word_tokenize(str(doc).lower()))

# Create a dictionary and corpus for the tokenized positive reviews
dictionary_pos = corpora.Dictionary(data['Tokenized_Positive_Review'])
corpus_pos = [dictionary_pos.doc2bow(doc) for doc in data['Tokenized_Positive_Review']]

# Function to get the top 7 topics with probabilities for each document
def get_top_topics(lda_model, corpus, num_topics=7):
    top_topics = []
    for doc in corpus:
        topics = lda_model.get_document_topics(doc)
        topics_sorted = sorted(topics, key=lambda x: x[1], reverse=True)[:num_topics]
        top_topics.append([(topic[0], topic[1]) for topic in topics_sorted])
    return top_topics

In [ ]:

# Assuming lda_model_10_topics is your trained LDA model
data['Top_Topics_Pos'] = get_top_topics(lda_model_10_topics, corpus_pos)

# Assuming topic_labels is your dictionary mapping topic numbers to labels
data['Top_Topics_Labels_Pos'] = data['Top_Topics_Pos'].apply(lambda topics: [topic_labels.get(topic[0], "Unknown Topic") for topic in topics])


In [ ]:
region_list = data["region"].unique().tolist()
for region in region_list:
    region_data = data[data['region'] == region]

    # Create a dictionary for scoring
    score_dict_pos = {}

    # Calculate the scores for each topic in the region
    for topics_list in region_data['Top_Topics_Labels_Pos']:
        for i, topic in enumerate(topics_list):
            if topic not in score_dict_pos:
                score_dict_pos[topic] = 0
            score_dict_pos[topic] += (7 - i)  # Adjusted score calculation for top seven topics

    # Sort the scores and find the top seven topics
    sorted_topics = sorted(score_dict_pos.items(), key=lambda x: (x[1], x[0]), reverse=True)[:7]

    # Output the top seven topics for the region
    print(f"Top seven topics of Positive Reviews for {region}:")
    for topic, score in sorted_topics:
        print(f"{topic}: {score}")
    print("\n")


Top seven topics of Positive Reviews for Europe:
Customer Services: 2079010
Easy Process: 1374864
Noise and Comfort: 1347279
Meals: 1340023
Location: 834862
Cleanliness: 743113
Facilities: 679701


Top seven topics of Positive Reviews for Oceania:
Customer Services: 146363
Easy Process: 98050
Meals: 97239
Noise and Comfort: 94012
Location: 57848
Cleanliness: 52129
Facilities: 40203


Top seven topics of Positive Reviews for Americas:
Customer Services: 281553
Easy Process: 194688
Meals: 188565
Noise and Comfort: 185775
Location: 108719
Cleanliness: 101623
Facilities: 86751


Top seven topics of Positive Reviews for Asia:
Customer Services: 385220
Easy Process: 312153
Noise and Comfort: 273448
Meals: 254600
Cleanliness: 165278
Facilities: 162129
Location: 160519


Top seven topics of Positive Reviews for Africa:
Customer Services: 50173
Easy Process: 37060
Noise and Comfort: 34515
Meals: 32034
Location: 20389
Facilities: 19520
Cleanliness: 19167


Top seven topics of Positive Reviews fo

In [ ]:
sub_region_list = data["sub_region"].unique().tolist()
for sub_region in sub_region_list:
    sub_region_data = data[data['sub_region'] == sub_region]

    # Create a dictionary for scoring
    score_dict_pos = {}

    # Calculate the scores for each topic in the sub-region
    for topics_list in sub_region_data['Top_Topics_Labels_Pos']:
        for i, topic in enumerate(topics_list):
            if topic not in score_dict_pos:
                score_dict_pos[topic] = 0
            score_dict_pos[topic] += (7 - i)  # Adjusted score calculation for top seven topics

    # Sort the scores and find the top seven topics
    sorted_topics = sorted(score_dict_pos.items(), key=lambda x: (x[1], x[0]), reverse=True)[:7]

    # Output the top seven topics for the sub-region
    print(f"Top seven topics of Positive Reviews for {sub_region}:")
    for topic, score in sorted_topics:
        print(f"{topic}: {score}")
    print("\n")


Top seven topics of Positive Reviews for Eastern Europe:
Customer Services: 107828
Easy Process: 80718
Noise and Comfort: 74038
Meals: 72167
Location: 44932
Cleanliness: 44282
Facilities: 37745


Top seven topics of Positive Reviews for Northern Europe:
Customer Services: 1603754
Easy Process: 1032852
Noise and Comfort: 1028366
Meals: 1025004
Location: 639180
Cleanliness: 559376
Facilities: 515970


Top seven topics of Positive Reviews for Australia and New Zealand:
Customer Services: 145499
Easy Process: 97480
Meals: 96634
Noise and Comfort: 93487
Location: 57480
Cleanliness: 51829
Facilities: 39945


Top seven topics of Positive Reviews for Western Europe:
Customer Services: 241813
Easy Process: 170573
Meals: 161755
Noise and Comfort: 159531
Location: 99715
Cleanliness: 89925
Facilities: 80436


Top seven topics of Positive Reviews for Southern Europe:
Customer Services: 125615
Easy Process: 90721
Noise and Comfort: 85344
Meals: 81097
Location: 51035
Cleanliness: 49530
Facilities: 45

In [ ]:
data[["Positive_Review_2", "Top_Topics_Pos", "Top_Topics_Labels_Pos"]].head(10)

,Positive_Review_2,Top_Topics_Pos,Top_Topics_Labels_Pos
0,"[park, outside, beautiful]","[(8, 0.4656045), (4, 0.26420465), (9, 0.192398...","[Easy Process, Customer Services, Noise and Co..."
1,"[real, complaint, location, surroundings, amen...","[(9, 0.3381056), (4, 0.25318256), (8, 0.180592...","[Noise and Comfort, Customer Services, Easy Pr..."
2,"[location, staff, ok, cute, breakfast, range, ...","[(8, 0.3354647), (4, 0.29399502), (3, 0.207109...","[Easy Process, Customer Services, Meals, Noise..."
3,"[location, surroundings, bar, restaurant, outd...","[(8, 0.3362651), (4, 0.17547598), (9, 0.136365...","[Easy Process, Customer Services, Noise and Co..."
4,"[location, building, romantic, setting]","[(7, 0.3500131), (4, 0.32061303), (8, 0.212706...","[Value, Customer Services, Easy Process, Noise..."
5,"[restaurant, modern, design, chill, park, near...","[(9, 0.4401033), (8, 0.2572065), (0, 0.1494226...","[Noise and Comfort, Easy Process, Location, Cu..."
6,"[spacious, bright, located, quiet, beautiful, ...","[(4, 0.4258423), (8, 0.28729737), (3, 0.236808...","[Customer Services, Easy Process, Meals]"
7,"[location, set, park, friendly, staff, food, h...","[(4, 0.57311565), (9, 0.2239613), (3, 0.086177...","[Customer Services, Noise and Comfort, Meals, ..."
8,[],"[(4, 0.54996616), (2, 0.050019223), (9, 0.0500...","[Customer Services, Facilities, Noise and Comf..."
9,"[big, enough, bed, breakfast, food, service, o...","[(4, 0.38471362), (8, 0.18335861), (3, 0.13279...","[Customer Services, Easy Process, Meals, Facil..."
